In [70]:
import pandas as pd
import numpy as np
import re
import datetime

data = pd.read_excel('C:\Users\Valery\Desktop\plan_escape\data\search_060916.xlsx')
dt = datetime.date(2016, 9, 11)

def first_number(x):
    x = re.sub(' +', '', x)
    if x == 'NULL':
        return -1
    
    else:
        return int(re.findall('^\d+', x)[0])
   
    
def text_string(x):
    x = re.sub(' +', '', x)
    if x == 'NULL':
        return 'NULL'
    else:
        return re.findall('^\d+(\D+)', x)[0] 

def phone_num(x):
    phone_list = re.findall('\d.\d{3}.\d{3}.\d{2}.\d{2}',x)
    phone_list = [re.sub('\D','',x) for x in phone_list]
    if len(phone_list) == 0:
        return [-1,-1]
    else:
        phone_list = [int(x) for x in phone_list]
    if len(phone_list) == 1:
        phone_list.append(-1)
    return phone_list

def time_posting(x):
    if pd.isnull(x):
        return datetime.datetime(3000,1,1,0,0)
    hour = int(re.findall(', (\d+):\d+',x)[0])
    minute = int(re.findall(', \d+:(\d+)',x)[0])
    s = re.split(',', x)[0]
    if s == u'вчера':
        dt_tmp = dt + datetime.timedelta(-1)
        day = dt_tmp.day
        month = dt_tmp.month
    elif s == u'сегодня': 
        day = dt.day
        month = dt.month
    else:
        s = re.sub(u'янв', '01', s.lower())
        s = re.sub(u'фев', '02', s.lower())
        s = re.sub(u'мар', '03', s.lower())
        s = re.sub(u'апр', '04', s.lower())
        s = re.sub(u'май', '05', s.lower())
        s = re.sub(u'июн', '06', s.lower())
        s = re.sub(u'июл', '07', s.lower())
        s = re.sub(u'авг', '08', s.lower())
        s = re.sub(u'сен', '09', s.lower())
        s = re.sub(u'окт', '10', s.lower())
        s = re.sub(u'ноя', '11', s.lower())
        s = re.sub(u'дек', '12', s.lower())
        day = int(re.findall('(\d+) \d+', s)[0])
        month = int(re.findall('\d+ (\d+)', s)[0])
    year = datetime.date.today().year
    return datetime.datetime(year,month,day,hour,minute)

#cnt_rooms = data.cnt_rooms.apply(lambda x: int(re.findall('^\d',x)[0]))
#2*x if x < 10 else raise_(Exception('foobar'))
cnt_rooms = data.cnt_rooms.apply(lambda x: -1 if len(re.findall('^\d',x))==0
                                                 else int(re.findall('^\d',x)[0]))
flat_ids = data.url.apply(lambda x: -1 if len(re.findall('/(\d+)/',x))==0
                                        else int(re.findall('/(\d+)/',x)[0]))
#data.dt_metro.fillna('NULL',inplace=True)
#dt_metro_int = data.dt_metro.apply(first_number)
dt_metro_int = data.dt_metro.apply(lambda x: -1 if len(re.findall('^\d+', re.sub(' +', '', x)))==0
                                                else int(re.findall('^\d+', re.sub(' +', '', x))[0]))
#dt_metro_type =  data.dt_metro.apply(text_string)
dt_metro_type =  data.dt_metro.apply(lambda x: "NULL" if len(re.findall('^\d+(\D+)', x))==0
                                                         else re.findall('^\d+(\D+)', x)[0])
#popularity_all = data.popularity.apply(lambda x: int(re.findall(u'всего (\d+)',x)[0]))
data.popularity.fillna("NULL",inplace=True)
popularity_all = data.popularity.apply(lambda x: -1 if len(re.findall(u'всего (\d+)',x))==0 
                                                       else int(re.findall(u'всего (\d+)',x)[0]))
#popularity_today = data.popularity.apply(lambda x: int(re.findall(u'сегодня (\d+)',x)[0]))
popularity_today = data.popularity.apply(lambda x: -1 if len(re.findall(u'сегодня (\d+)',x))==0
                                                         else int(re.findall(u'сегодня (\d+)',x)[0]))
#floor_cur = data.floor.apply(lambda x: int(re.split('/',x)[0]))
#floor_all = data.floor.apply(lambda x: int(re.split('/',x)[1]))
floor_cur = data.floor.apply(lambda x: -1 if len(re.findall('/',x))==0
                                                         else int(re.split('/',x)[0]))
floor_all = data.floor.apply(lambda x: -1 if len(re.findall('/',x))==0
                                                         else int(re.split('/',x)[1]))


#living_area = data.living_area.apply(lambda x: float(re.sub(u'\u2013','-1',re.sub(',','.',re.sub(u'м2','',x)))))
#kitchen_area = data.kitchen_area.apply(lambda x: float(re.sub(u'\u2013','-1',re.sub(',','.',re.sub(u'м2','',x)))))
living_area = data.living_area.apply(lambda x: -1 if len(re.findall("\D",re.sub(u'\u2013','-1',re.sub(',','.',re.sub(u'м2','',f)))))>1
                                                         else float(re.sub(u'\u2013','-1',re.sub(',','.',re.sub(u'м2','',x)))))
kitchen_area = data.kitchen_area.apply(lambda x: -1 if len(re.findall("\D",re.sub(u'\u2013','-1',re.sub(',','.',re.sub(u'м2','',f)))))>1
                                                         else float(re.sub(u'\u2013','-1',re.sub(',','.',re.sub(u'м2','',x)))))

for f in ['avg_price_m2_in_house',
          'avg_rental_rate',
          'district_population',
          'avg_age',
          'avg_price_m2_in_district']:
    data[f].fillna('0',inplace=True)
avg_price_m2_in_house = data.avg_price_m2_in_house.apply(lambda x: re.sub('\D','',x))
avg_rental_rate = data.avg_rental_rate.apply(lambda x: re.sub('\D','',x))
district_population = data.district_population.apply(lambda x: re.sub('\D','',x))
avg_age = data.avg_age.apply(lambda x: re.sub('\D','',x))
avg_price_m2_in_district = data.avg_price_m2_in_district.apply(lambda x: re.sub('\D','',x))
phone_first = [x[0] for x in data.message.apply(phone_num)]
phone_second = [x[1] for x in data.message.apply(phone_num)]

data_clean = pd.DataFrame(data=zip(flat_ids,
                          cnt_rooms,
                          data.total_area,
                          kitchen_area,
                          living_area,
                          floor_cur,
                          floor_all,
                          data.price_all,
                          np.round((data.price_all/data.total_area).values),
                          avg_price_m2_in_house,
                          avg_price_m2_in_district,
                          avg_rental_rate,
                          popularity_today,
                          popularity_all,
                          data.metro,
                          dt_metro_int,
                          dt_metro_type,
                          data.year_of_construction,
                          data.house_type,
                          data.cnt_entrance,
                          data.cnt_flats,
                          data.name_of_district,
                          district_population,
                          avg_age,
                          phone_first,
                          phone_second,
                          data.time_of_posting.apply(time_posting),           
                          [dt for i in range(data.shape[0])]  
                         ),
                 columns = ['id',
                            'cnt_rooms',
                            'total_area',
                            'kitchen_area',
                            'living_area',
                            'floor_cur',
                            'floor_all',
                            'price_all',
                            'price_m2',
                            'avg_price_m2_in_house',
                            'avg_price_m2_in_district',
                            'avg_rental_rate',
                            'popularity_today',
                            'popularity_all',
                            'metro',
                            'metro_time',
                            'metro_time_type',
                            'year_of_construction',
                            'house_type',
                            'cnt_entrance',
                            'cnt_flats',
                            'name_of_district',
                            'district_population',
                            'avg_age_of_constructions',
                            'phone_first',
                            'phone_second',
                            'dt_post',
                            'dt_report'
                           ]
                )

In [71]:
data_clean

,id,cnt_rooms,total_area,kitchen_area,living_area,floor_cur,floor_all,price_all,price_m2,avg_price_m2_in_house,...,house_type,cnt_entrance,cnt_flats,name_of_district,district_population,avg_age_of_constructions,phone_first,phone_second,dt_post,dt_report
0,149006545,1,32.0,-1,-1,4,9,5100000,159375.0,153874,...,панельный,10.0,356.0,Царицыно,127911,45,79268269237,79683209900,2016-09-05 07:49:00,2016-09-11
1,147183006,2,37.0,-1,-1,1,9,6200000,167568.0,210286,...,блочный,1.0,70.0,Таганский,119094,55,79265336793,79154665002,2016-09-05 09:20:00,2016-09-11
2,148476082,1,41.0,-1,-1,2,14,7472298,182251.0,0,...,NaN,NaN,NaN,NaN,0,0,74951342651,-1,3000-01-01 00:00:00,2016-09-11
3,148246400,2,48.0,-1,-1,5,9,6500000,135417.0,145266,...,панельный,6.0,384.0,Люблино,170143,41,79160135063,-1,2016-08-31 07:24:00,2016-09-11
4,149199027,2,60.0,-1,-1,2,14,6611000,110183.0,0,...,NaN,NaN,NaN,NaN,0,0,74952414506,-1,2016-09-02 17:04:00,2016-09-11
5,148220620,2,45.0,-1,-1,8,9,7000000,155556.0,153833,...,панельный,6.0,209.0,Орехово-Борисово Северное,131409,36,79150313275,-1,2016-09-02 11:01:00,2016-09-11
6,149025625,1,35.0,-1,-1,2,5,6100000,174286.0,148572,...,панельный,4.0,80.0,Люблино,170143,41,79263110554,74957450518,2016-09-02 17:03:00,2016-09-11
7,149087671,-1,41.0,-1,-1,5,14,7578491,184841.0,0,...,NaN,NaN,NaN,NaN,0,0,74951346540,-1,3000-01-01 00:00:00,2016-09-11
8,149383269,2,43.0,-1,-1,3,5,6990000,162558.0,151583,...,кирпичный,4.0,76.0,Ивановское,129814,41,79031838001,74959253929,2016-09-04 13:47:00,2016-09-11
9,149391436,1,33.0,-1,-1,3,9,6550000,198485.0,180826,...,кирпичный,3.0,108.0,Южнопортовый,73178,56,79251288166,-1,2016-09-04 16:56:00,2016-09-11
